#### Cemetery check pictures in SPA och söka i SPA
* [This notebook](https://github.com/salgo60/spa2Commons/blob/main/Notebook/SPA%20IFKDB.ipynb) 


In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2023-08-02 10:39:27.835420


In [2]:
import urllib3, json
import pandas as pd 
http = urllib3.PoolManager() 

url= "https://portrattarkiv.se/endpoints/search.php"  


In [3]:
# wd:Q26257009. Gamla Gävle Behravningsplats
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

# IFKDB personer födda innan 1920 - SPA innehåller m,est gamla personer
queryIFKDB = """SELECT (CONCAT (?itemLabel," ",str(year(?birthDate))) AS ?search) ?item ?itemLabel 
  (str(year(?birthDate)) AS ?year)
  WHERE {
  ?item wdt:P11905 ?ifkdb. 
  ?item wdt:P569 ?birthDate. 
  FILTER(xsd:integer(year(?birthDate)) < 1920)
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv", "en". }
} order by ?itemLabel """


 
def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

#SPAdf = get_sparql_dataframe(endpoint_url, query)
#SPAdf = get_sparql_dataframe(endpoint_url, queryNoPicNorra1890)
SPAdf = get_sparql_dataframe(endpoint_url, queryIFKDB)

SPAdf.info()   

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   search     61 non-null     object
 1   item       61 non-null     object
 2   itemLabel  61 non-null     object
 3   year       61 non-null     object
dtypes: object(4)
memory usage: 2.0+ KB


In [4]:
  SPAdf

,search,item,itemLabel,year
0,Alf Hjelm 1902,http://www.wikidata.org/entity/Q116600700,Alf Hjelm,1902
1,Allan Mattiasson 1908,http://www.wikidata.org/entity/Q116112391,Allan Mattiasson,1908
2,Anders Bernmar 1917,http://www.wikidata.org/entity/Q5574006,Anders Bernmar,1917
3,Anders Rydberg 1903,http://www.wikidata.org/entity/Q491262,Anders Rydberg,1903
4,Arne Nyberg 1913,http://www.wikidata.org/entity/Q1484638,Arne Nyberg,1913
...,...,...,...,...
56,Verner Jansson 1890,http://www.wikidata.org/entity/Q121056331,Verner Jansson,1890
57,Verner Johansson 1886,http://www.wikidata.org/entity/Q121075959,Verner Johansson,1886
58,Walter Lidén 1887,http://www.wikidata.org/entity/Q5950548,Walter Lidén,1887
59,Åke Hallman 1912,http://www.wikidata.org/entity/Q10726271,Åke Hallman,1912


In [5]:
SPAdetail = "https://portrattarkiv.se/details/"
for index, row  in SPAdf.iterrows():
    print("\n",row["search"],row["item"])
    encoded_body = json.dumps({
        "limit": "5",
        "from": "0",
        "birthyear":row["year"],
        "all":row["search"]
    })
    
    r = http.request('POST', url,
                 headers={'Content-Type': 'application/json'},
                 body=encoded_body)

    if r.status != 200:
        print(r.status)
        continue
    
    data = json.loads(r.data.decode('utf-8'),) 
    urls = []


    for h in data["hits"]["hits"]:
        id = h["_id"]
        #print(h)
        source = h["_source"]
        try:
 #           urlPicture = urlbasePic + id
 #           urls.append(urlPicture)
            score = h["_score"]
            FirstName = source["FirstName"]
            LastName = source["LastName"]
            BirthYear = source["BirthYear"]
            print("\t\t",score,FirstName, " ", LastName, " - ", BirthYear,SPAdetail+id, "\t", )
        except:
            print("Error")



 Alf Hjelm 1902 http://www.wikidata.org/entity/Q116600700
		 19.885498 Evert   Hjelm  -  1902 https://portrattarkiv.se/details/j6S8rJvx9lAAAAAAAACrgw 	
		 19.55274 Gunvor   Hjelm-Höjer  -  1902 https://portrattarkiv.se/details/j6S8rJvx9lAAAAAAAABY4Q 	
		 18.449177 Yngve   Hjelm  -  1902 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABymPA 	
		 18.415318 Gösta   Hjelm  -  1902 https://portrattarkiv.se/details/j6S8rJvx9lAAAAAAAABX8Q 	
		 18.023052 Filip   Hjelm  -  1902 https://portrattarkiv.se/details/seuNpYu6DKAAAAAAAABsZQ 	

 Allan Mattiasson 1908 http://www.wikidata.org/entity/Q116112391
		 23.026072 Johan   Mattiasson  -  1908 https://portrattarkiv.se/details/5TJc-sPXaKAAAAAAAABdMw 	
		 22.33537 Johan   Mattiasson  -  1908 https://portrattarkiv.se/details/I_2wtd5VefAAAAAAAABPBw 	
		 16.532497 Allan   Andersson  -  1908 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAA5cSg 	
		 16.4977 Allan   Birgersson  -  1908 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAA2PsA 	
		 16.2

		 20.314 Karl Arnold Douglas   Larsson  -  1897 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAACiN4A 	
		 19.261642 Albert (Abbe) Ragnar   Krook  -  1897 https://portrattarkiv.se/details/0piq-fX0h5AAAAAAAABkjw 	
		 19.255539 Douglas   Olsson  -  1897 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAACet2g 	
		 18.228987 Douglas   Bondeman  -  1897 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAA2JNg 	
		 18.155624 Douglas   Wallhäll  -  1897 https://portrattarkiv.se/details/ciLMGSqrYjAAAAAAAABicA 	

 Einar Andersson 1885 http://www.wikidata.org/entity/Q121075908
		 16.200027 Einar   Karlberg  -  1885 https://portrattarkiv.se/details/0piq-fX0h5AAAAAAAAAo5w 	
		 16.131756 Einar   Sundström  -  1885 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABJu4Q 	
		 16.12999 Einar   Björe  -  1885 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAA2aJw 	
		 16.04565 Einar   Nyman  -  1885 https://portrattarkiv.se/details/wbbNj8LLWMAAAAAAAABWew 	
		 16.008652 Einar   Strand  -  1885 https:

		 17.504532 Gustav   Johansson  -  1914 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABVYDA 	
		 17.503242 Gustav   Johansson  -  1914 https://portrattarkiv.se/details/Di4a42b6X_AAAAAAAACbdQ 	
		 17.450855 Gustav   Johansson  -  1914 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABNESQ 	
		 17.440922 Gustav   Johansson  -  1914 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABWX_Q 	
		 17.220078 Gustav   Johansson  -  1914 https://portrattarkiv.se/details/Di4a42b6X_AAAAAAAAAu4g 	

 Harry Hjelm 1898 http://www.wikidata.org/entity/Q121008355
		 20.322289 Emil   Hjelm  -  1898 https://portrattarkiv.se/details/j6S8rJvx9lAAAAAAAACp-A 	
		 18.388964 Ruben   Hjelm  -  1898 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABVWVg 	
		 18.03904 Erik   Hjelm  -  1898 https://portrattarkiv.se/details/j6S8rJvx9lAAAAAAAABYCA 	
		 17.989208 Albert   Hjelm  -  1898 https://portrattarkiv.se/details/j6S8rJvx9lAAAAAAAABYQA 	
		 17.35972 Robert   Hjelm  -  1898 https://portrattarkiv.se/details/j

		 17.48937 Edvin   Sandberg  -  1895 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAACYrYA 	
		 17.316572 Edvin   Sandberg  -  1895 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAACet7w 	
		 17.289211 Nils   Sandberg  -  1895 https://portrattarkiv.se/details/4e1scVvZ-EAAAAAAAAA46A 	
		 17.21621 John   Sandberg  -  1895 https://portrattarkiv.se/details/CaU9ScYzgeAAAAAAAAADJQ 	
		 17.122978 Vera   Resare-Sandberg  -  1895 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAACemMA 	

 Nils Andersson 1887 http://www.wikidata.org/entity/Q1992244
		 17.361877 Nils   Andersson  -  1887 https://portrattarkiv.se/details/28K7pO-SRrAAAAAAAACqdw 	
		 17.361877 Nils   Andersson  -  1887 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABhGPg 	
		 16.37098 Nils   Agell  -  1887 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABKfJw 	
		 16.361233 Nils   Andersson  -  1887 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAA5GIg 	
		 15.825604 Nils   Annertz  -  1887 https://portrattarkiv.se/detail